In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [4]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.2

  Using cached crewai-0.28.8-py3-none-any.whl.metadata (13 kB)
  Using cached crewai_tools-0.1.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached langchain_community-0.0.2-py3-none-any.whl.metadata (5.8 kB)
  Using cached instructor-0.5.2-py3-none-any.whl.metadata (10 kB)
  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
  Using cached regex-2023.12.25-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
  Using cached lancedb-0.5.7-py3-none-any.whl.metadata (17 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached chroma-hnswlib-0.7.3.tar.gz (31 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to buil

ERROR: Cannot install crewai and langchain_community==0.0.2 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [6]:
import os

In [7]:
from crewai import LLM
llm = LLM(
    model="ollama/llama3.1:8b",
    base_url="http://localhost:11434"
)


In [8]:
tavily_api_key="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"


In [9]:
from crewai_tools import (
 FileReadTool,
 ScrapeWebsiteTool,
 MDXSearchTool,
 SerperDevTool
 )

In [15]:
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ["SERPER_API_KEY"]="7d36b988d4d42221ffbbca57e5ec34a3d32de897"

In [17]:
search_tool=SerperDevTool()
scrape_tool=ScrapeWebsiteTool()
read_resume=FileReadTool(file_path="./Resume.pdf")


In [19]:
scrape_tool

ScrapeWebsiteTool(name='Read website content', description="Tool Name: Read website content\nTool Arguments: {'website_url': {'description': 'Mandatory website url to read the file', 'type': 'str'}}\nTool Description: A tool that can be used to read a website content.", args_schema=<class 'crewai_tools.tools.scrape_website_tool.scrape_website_tool.ScrapeWebsiteToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x000002758A2500E0>, result_as_answer=False, website_url=None, cookies=None, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9', 'Accept-Language': 'en-US,en;q=0.9', 'Referer': 'https://www.google.com/', 'Connection': 'keep-alive', 'Upgrade-Insecure-Requests': '1'})

In [21]:
search_tool

SerperDevTool(name='Search the internet', description="Tool Name: Search the internet\nTool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}\nTool Description: A tool that can be used to search the internet with a search_query.", args_schema=<class 'crewai_tools.tools.serper_dev_tool.serper_dev_tool.SerperDevToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x000002758A2500E0>, result_as_answer=False, search_url='https://google.serper.dev/search', country='', location='', locale='', n_results=10, save_file=False)

In [23]:
from crewai_tools import PDFSearchTool
import embedchain


print("embedchain version:", embedchain.__version__)

semantic_search_resume = PDFSearchTool(
    config=dict(
        llm=dict(
            provider="ollama",
            config=dict(
                model="llama3.1:8b",
            ),
        ),
        embedder=dict(
            provider="ollama",
            config=dict(
                model="nomic-embed-text",
            ),
        ),
    )
)


embedchain version: 0.1.125


C:\Users\SUN\anaconda3\Lib\site-packages\embedchain\embedder\ollama.py:27: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=self.config.model, base_url=config.base_url)


In [ ]:
!pip install embedchain

In [ ]:
!pip install google-generativeai


In [24]:
 researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
)
 )

In [27]:
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do incredible research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [29]:
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [31]:
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

In [33]:
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    llm=llm,
    agent=researcher,
    async_execution=True
)

In [35]:
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using only the exact GitHub  URL {{github_url}}, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [37]:
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [39]:
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


In [41]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

In [43]:
job_application_inputs = {
    'job_posting_url': 'https://wellfound.com/jobs?job_listing_id=3162817',
    'github_url': 'https://github.com/Revanth-shivakumar',
    'personal_writeup': """Revanth is a budding software engineer with good combination of development skills"""
}

In [45]:
result = job_application_crew.kickoff(inputs=job_application_inputs)


# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://wellfound.com/jobs?job_listing_id=3162817) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
# Agent: Personal Profiler for Engineers
## Task: Compile a detailed personal and professional profile using only the exact GitHub  URL {github_url}, and personal write-up (Revanth is a budding software engineer with good combination of development skills). Utilize tools to extract and synthesize information from these sources.


# Agent: Tech Job Researcher
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://wellfound.com/jobs?job_listing_id=3162817\"}"
## Tool Output: 
wellfound.com Please enable JS and disable any ad blocker


# Agent: Tech Job Researcher
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"how to read wellfound.com job posting\"}"
## Tool Output: 

Sea